<a href="https://colab.research.google.com/github/filosoficaguilar/Coursera_Capstone/blob/main/Copia_de_Neighborhoods_in_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geocoder

In [ ]:
#Import libraries
import pandas as pd
import numpy as np
import requests
import geocoder
from bs4 import BeautifulSoup

<h1>Excercise 1: Getting Data and Creating DataFrame</h1>

In [ ]:
#request to wikipedia by method get
document = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
#parsing the response text to html format
soup = BeautifulSoup(document.text, 'html.parser')
#extracting the table on html formatt o list with all td
list_table= soup.table.find_all("td")

#filling raw_data with the text of each td
raw_data = [i.text.replace("\n","") for i in list_table]

#spliting the data into slices
data_2 = [raw_data[x:x+3] for x in range(0, len(raw_data), 3)]

#dropping the Not Assigned data
final_data = []
for i in data_2:
  if(i[1] != "Not assigned"):
    final_data.append(i)
final_data
df = pd.DataFrame(data=final_data,columns=["Postal Code","Borough","Neighborhood"])
df.shape

<h1>Excercise 2: Getting Geolocalization (by CSV file)</h1>

In [ ]:
postal_codes = pd.read_csv("http://cocl.us/Geospatial_data")
df_geo = pd.merge(df, postal_codes)
df_geo



<h1>Execise 3: Map And Klustering</h1>


In [ ]:
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 

In [ ]:
df_toronto = df_geo[df_geo['Borough'].str.contains('Toronto',regex=False)]
df_toronto

In [ ]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(df_toronto['Latitude'],df_toronto['Longitude'],df_toronto['Borough'],df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

In [ ]:
k=5
toronto_clustering = df_toronto.drop(['Postal Code','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
df_toronto.insert(0, 'Cluster Labels', kmeans.labels_)

In [ ]:
df_toronto

In [ ]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighborhood'], df_toronto['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters
